In [58]:
from pycozo.client import Client

In [380]:
# read docs
paths = [
    r"../assets/text/da-lec1-notes.txt",
    r"../assets/text/da-lec2-notes.txt",
    r"../assets/text/da-lec3-notes.txt",
    r"../assets/text/da-lec4-notes.txt",
    r"../assets/text/da-lec5-notes.txt",
    r"../assets/text/da-lec6-notes.txt",
    r"../assets/text/da-lec7-notes.txt",
    r"../assets/text/da-lec8-notes.txt",
    r"../assets/text/da-lec9-notes.txt",
    r"../assets/text/da-lec10-notes.txt",
    r"../assets/text/da-lec11-notes.txt",
    r"../assets/text/da-lec12-notes.txt",
]
from pathlib import Path

paths = [Path(p) for p in paths]

In [381]:
import pandas as pd

document_relation = pd.DataFrame(
    columns=["document_name","document_path"],
    data=[(path.name, path) for i, path in enumerate(paths)]
)


In [382]:
client = Client('sqlite', 'file.db')

First create a document path - document id relation

In [383]:
client.run(
    """
    :create document {document_name => document_path}
    """
)

,status
0,OK


In [384]:
for document_name, document_path in document_relation.values:
    client.run("""
    ?[document_name, document_path] <- [[$document_name, $document_path]]
    :put document {document_name => document_path}
    """,{"document_name":str(document_name),"document_path":str(document_path)})


In [385]:
client.run("""
    :create sentence {sentence_id, paragraph_id, doc_name => sentence} 
""")

,status
0,OK


In [386]:
client.run("""
    :create phrases {phrase:String, doc_name:String, paragraph_id:Int?}
""")

,status
0,OK


In [387]:
# read phrases 
import json
with open(r"../data/results_decision_analysis_slides.json","r") as f:
    phrases = json.load(f)
phrases = {k+".txt":v for k,v in phrases.items()}
phrases.keys()

dict_keys(['da-lec1-notes.txt', 'da-lec2-notes.txt', 'da-lec3-notes.txt', 'da-lec4-notes.txt', 'da-lec5-notes.txt', 'da-lec6-notes.txt', 'da-lec7-notes.txt', 'da-lec8-notes.txt', 'da-lec9-notes.txt', 'da-lec10-notes.txt', 'da-lec11-notes.txt', 'da-lec12-notes.txt'])

In [388]:
from tqdm import tqdm

In [389]:
sentence_dict = {
    "sentence": {
        "headers": ["sentence_id", "paragraph_id", "doc_name", "sentence"],
        "rows": [],
    }
}
for path in paths:
    with open(path, 'r') as f:
        paragraphs = f.read().splitlines()
    doc_name = path.name
    doc_path = path
    for paragraph_id,paragraph in enumerate(paragraphs):
        if len(paragraph) > 0:
            sents = sent_tokenize(paragraph)
            for sentence_id, sent in enumerate(sents):
                sentence_dict["sentence"]["rows"].append(
                    [sentence_id, paragraph_id, doc_name, sent]
                )

# sentence_dict
client.import_relations(sentence_dict)

# tx.commit()


In [390]:
client.run("""
    ::fts create sentence:phrases {
    extractor: sentence,
    tokenizer: Simple,
    filters: [Lowercase,AlphaNumOnly, Stemmer('English')],
}
""")

,status
0,OK


In [391]:
phrase_dict = {
    "phrases": {
        "headers": ["phrase", "doc_name", "paragraph_id"],
        "rows": []
    }
}
all_phrases = [phrase for doc_phrases in phrases.values() for phrase in doc_phrases]
all_phrases = list(set(all_phrases))
for original_phrase in tqdm(all_phrases,desc="phrases",total=len(all_phrases)):
    #replace non alphanumeric characters with space
    phrase = "".join([c if c.isalnum() else " " for c in original_phrase])
    # split phrase into 2 word spans
    phrase = phrase.split()
    span = " ".join(phrase[0:2])

    total = client.run("""
        ?[paragraph_id, doc_name] := ~sentence:phrases {paragraph_id,doc_name, sentence |
            query: $query,
            k: 1000
        }
        """, {"query": f"NEAR/1({span})"})
    
    for i in range(1,len(phrase)-1):
        span = " ".join(phrase[i:i+2])
        results = client.run("""
        ?[paragraph_id, doc_name] := ~sentence:phrases {paragraph_id,doc_name, sentence |
            query: $query,
            k: 1000
        }
        """, {"query": f"NEAR/1({span})"})
        total = pd.merge(total,results,how="inner")
    for paragraph_id, doc_name in total.values:
        phrase_dict["phrases"]["rows"].append([original_phrase, doc_name, paragraph_id])
        
client.import_relations(phrase_dict)

phrases: 100%|██████████| 1984/1984 [00:08<00:00, 225.66it/s]


In [405]:
client.run("""
    ?[doc_name, paragraph_id] := *phrases["worst class", doc_name, paragraph_id]
    """)

,doc_name,paragraph_id
0,da-lec2-notes.txt,3
1,da-lec2-notes.txt,4
2,da-lec2-notes.txt,30
3,da-lec3-notes.txt,35


In [412]:
notes_path = Path(r"C:\Users\kryst\Documents\Personal\university\ai_sem6\nlp\testing")
notes_path

WindowsPath('C:/Users/kryst/Documents/Personal/university/ai_sem6/nlp/testing')

In [426]:
original_phrase

'binary classification'

In [465]:
def get_paragraphs(original_phrase):
    dataframe = client.run("""
        paragraphs[doc_name, paragraph_id] := *phrases[$phrase, doc_name, paragraph_id],
        ?[sentence,sentence_id,paragraph_id,doc_name] := *sentence[sentence_id, paragraph_id, doc_name, sentence],paragraphs[doc_name, paragraph_id]
        :order doc_name, paragraph_id, sentence_id
        """,{"phrase":original_phrase})
    # for each combination of paragraph_id and doc_name, get the paragraph
    return dataframe.groupby(["doc_name","paragraph_id"]).agg({"sentence":lambda x: " ".join(x)})
    
    
df = get_paragraphs("best class")

In [466]:
list(df.iterrows())

[(('da-lec2-notes.txt', 3),
  sentence     The basic idea underlying the sorting method ...
  Name: (da-lec2-notes.txt, 3), dtype: object),
 (('da-lec2-notes.txt', 32),
  sentence     Let us check the results of our study while m...
  Name: (da-lec2-notes.txt, 32), dtype: object),
 (('da-lec3-notes.txt', 35),
  sentence     The question is how to translate assignment e...
  Name: (da-lec3-notes.txt, 35), dtype: object)]

In [467]:
df.groupby(["doc_name","paragraph_id"]).all()

sentence
doc_name          paragraph_id          
da-lec2-notes.txt 3                 True
                  32                True
da-lec3-notes.txt 35                True

In [472]:
for original_phrase in tqdm(all_phrases, desc="notes", total=len(all_phrases)):
    md_text = ""
    paragraphs = get_paragraphs(original_phrase)
    for paragraph_description,paragraph in paragraphs.iterrows():
        md_text += f"\n\n `document : {paragraph_description[0]} slide: {paragraph_description[1]}`"
        md_text += f"\n\n {paragraph['sentence']}"
    md_text += "\n\n## See also:"
    adjacent = client.run("""
        ?[phrase] := *phrases[phrase, doc_name, paragraph_id], *phrases[$phrase, doc_name, paragraph_id], phrase!= $phrase
        """,{"phrase":original_phrase}).values
    for phrase in adjacent:
        md_text += f"\n[[{phrase[0]}]]"
    cleaned_phrase = "".join([c if c.isalnum() else " " for c in original_phrase])
    with open(notes_path / f"{cleaned_phrase}.md","w") as f:
        f.write(md_text)


notes: 100%|██████████| 1984/1984 [04:06<00:00,  8.06it/s]
